In [83]:
import numpy as np
import pandas as pd
import requests
import json
from time import sleep
from tqdm import tqdm

In [73]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#https://rosreestr.gov.ru/api/online/fir_object/25:28:10025:1044
#https://curlconverter.com/

In [74]:
#копируем cURL (bash), затем вставляем в https://curlconverter.com/, получем готовый код
def get_category(kn):
    cookies = {
        'session-cookie': '180f4028650d668362e6272ebeb261f519c98d63bdc38e88191e9caedbc6e15cc8e3b652f54f10f9f1f152d455d0a37c',
        'uid': 'CoHkfWdVx6sNHWDVE2YGAg==',
    }
    
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
        # 'Cookie': 'session-cookie=180f4028650d668362e6272ebeb261f519c98d63bdc38e88191e9caedbc6e15cc8e3b652f54f10f9f1f152d455d0a37c; uid=CoHkfWdVx6sNHWDVE2YGAg==',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }
    
    response = requests.get(
        f'https://rosreestr.gov.ru/api/online/fir_object/{kn}',
        cookies = cookies,
        headers = headers,
        verify = False,
        timeout = 10
    )

    return response.json()

In [76]:
def no_zerro(kn):
    parts = kn.split(":")
    cleaned_parts = [part.lstrip("0") or "0" for part in parts]
    #cleaned_number = ":".join(cleaned_parts)
    return ":".join(cleaned_parts)

In [86]:
kn = '77:06:0003016:2536'

no_zerro(kn)

'77:6:3016:2536'

In [48]:
kn = '25:28:10022:720'

response = get_category(kn)

print(response['objectData']['objectAddress']['mergedAddress'])
print(response['parcelData']['areaValue'])

г Владивосток, ул Славянская, д. 17, 140
50.2


C:\Users\skapr\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rosreestr.gov.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [88]:
df = pd.read_excel(r'мо.xlsx', sheet_name = 0)
df

,Unnamed: 0,Unnamed: 1
0,50:09:0070424:168,"Московская область, городской округ Солнечного..."
1,50:20:0020202:3956,"Московская область, Одинцовский район, раб. по..."
2,50:08:0050435:778,"143582, обл. Московская, р-н Истринский, д. За..."
3,50:23:0000000:117210,"140105, обл. Московская, г. Раменское, ш. Севе..."
4,50:23:0110103:665,"140105, обл. Московская, г. Раменское, ш. Севе..."
...,...,...
3654,50:14:0050283:2583,NaN
3655,50:14:0050283:2606,NaN
3656,50:14:0050412:734,NaN
3657,50:14:0050412:1067,NaN


In [89]:
filtered_df = df[df['Unnamed: 1'].isna()]
filtered_df.reset_index(drop = True, inplace = True)
filtered_df

,Unnamed: 0,Unnamed: 1
0,50:64:0000000:16852,NaN
1,50:64:0000000:16853,NaN
2,50:28:0010456:2069,NaN
3,50:28:0010456:1757,NaN
4,50:28:0050105:8391,NaN
...,...,...
1731,50:14:0050283:2583,NaN
1732,50:14:0050283:2606,NaN
1733,50:14:0050412:734,NaN
1734,50:14:0050412:1067,NaN


In [90]:
table = []

for kns in tqdm(filtered_df['Unnamed: 0']):
    kns_nz = no_zerro(kns)
    #print(kns_nz)
    try:
        response = get_category(kns_nz)
    
        table.append({
                      'Адрес': response['objectData']['objectAddress']['mergedAddress'],
                      'Площадь': response['parcelData']['areaValue']
                      })
    except:
        #pass
        table.append({
                      'Адрес': 0,
                      'Площадь': 0
                        })
#table

100%|██████████████████████████████████████████████████████████████████████████████| 1736/1736 [03:22<00:00,  8.57it/s]


In [91]:
df_pars = pd.DataFrame(table)

df_itog = pd.concat([filtered_df, df_pars], axis = 1)
df_itog

,Unnamed: 0,Unnamed: 1,Адрес,Площадь
0,50:64:0000000:16852,NaN,"г Дзержинский, ул Угрешская, д. 26б, 37",70.2
1,50:64:0000000:16853,NaN,"г Дзержинский, ул Угрешская, д. 26б, 38",47.5
2,50:28:0010456:2069,NaN,"г Домодедово, мкр Западный, ул Текстильщиков, ...",38.5
3,50:28:0010456:1757,NaN,"г Домодедово, мкр Западный, ул Текстильщиков, ...",58.6
4,50:28:0050105:8391,NaN,0,0.0
...,...,...,...,...
1731,50:14:0050283:2583,NaN,0,0.0
1732,50:14:0050283:2606,NaN,0,0.0
1733,50:14:0050412:734,NaN,0,0.0
1734,50:14:0050412:1067,NaN,0,0.0


In [92]:
#Сохраняем в Excel
writer = pd.ExcelWriter('мо_парс.xlsx')
df_itog.to_excel(writer, 'мо_парс')
writer._save()

# Фонд Реновации

In [ ]:
#https://fr.mos.ru/uchastnikam-programmy/karta-renovatsii/?ft=1&category[]=ALL

In [93]:
import requests

cookies = {
    'PHPSESSID': 'eCiW669y3jsxQvZG6epwc4Q8ZwwhE4Zv',
    'BITRIX_SM_PK': 'page',
    '_ym_uid': '1733942248514627063',
    '_ym_d': '1733942248',
    'BITRIX_CONVERSION_CONTEXT_s1': '%7B%22ID%22%3A2%2C%22EXPIRE%22%3A1733950740%2C%22UNIQUE%22%3A%5B%22conversion_visit_day%22%5D%7D',
    'BX_USER_ID': '6190a95436ff61c811c8f10618549d1c',
    '_ym_isad': '2',
    '_ym_visorc': 'w',
}

headers = {
    'accept': '*/*',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    # 'cookie': 'PHPSESSID=eCiW669y3jsxQvZG6epwc4Q8ZwwhE4Zv; BITRIX_SM_PK=page; _ym_uid=1733942248514627063; _ym_d=1733942248; BITRIX_CONVERSION_CONTEXT_s1=%7B%22ID%22%3A2%2C%22EXPIRE%22%3A1733950740%2C%22UNIQUE%22%3A%5B%22conversion_visit_day%22%5D%7D; BX_USER_ID=6190a95436ff61c811c8f10618549d1c; _ym_isad=2; _ym_visorc=w',
    'priority': 'u=1, i',
    'referer': 'https://fr.mos.ru/uchastnikam-programmy/karta-renovatsii/?ft=1&category[]=ALL',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

response = requests.get(
    'https://fr.mos.ru/pokupka-nedvizhimosti-dlya-vseh/ajax.php?category[]=ALL&pagesize=100000',
    cookies = cookies,
    headers = headers,
)

In [94]:
response.json()

{'objects': {'items': [{'id': '32698',
    'name': '1-й Ботанический проезд, дом 1',
    'code': '1-j-botanicheskij-proezd-dom-1',
    'district': 2556,
    'coords': ['55.8502', '37.634'],
    'status_code': 'OLD',
    'polygon': 'POLYGON((55.8499893843668 37.6338654756546,55.8499683047149 37.6340478658676,55.8504260317294 37.6342034339905,55.8504440997905 37.6340210437775,55.8499893843668 37.6338654756546))',
    'county': 2455},
   {'id': '32700',
    'name': '1-й Ботанический проезд, дом 3',
    'code': '1-j-botanicheskij-proezd-dom-3',
    'district': 2556,
    'coords': ['55.851', '37.6343'],
    'status_code': 'OLD',
    'polygon': 'POLYGON((55.8507904026702 37.6341390609741,55.8507723347701 37.6343214511871,55.8512300523153 37.63447701931,55.8512511312829 37.634294629097,55.8507904026702 37.6341390609741))',
    'county': 2455},
   {'id': '29325',
    'name': '1-й Ботанический проезд, дом 3А',
    'code': '1-j-botanicheskij-proezd-dom-3a',
    'district': 2556,
    'coords': ['

In [109]:
renovation_msk = response.json()

In [110]:
len(renovation_msk['objects']['items'])

5965

In [117]:
renovation_m = []

for i in range(len(renovation_msk['objects']['items'])):
    renovation_m.append({
                         'Адрес': renovation_msk['objects']['items'][i]['name'],
                         'lat': renovation_msk['objects']['items'][i]['coords'][0],
                         'lon': renovation_msk['objects']['items'][i]['coords'][1],
                         'status_code': renovation_msk['objects']['items'][i]['status_code']
                        })

In [118]:
df_ren = pd.DataFrame(renovation_m)
df_ren

,Адрес,lat,lon,status_code
0,"1-й Ботанический проезд, дом 1",55.8502,37.634,OLD
1,"1-й Ботанический проезд, дом 3",55.851,37.6343,OLD
2,"1-й Ботанический проезд, дом 3А",55.8511,37.6336,OLD
3,"1-й Волоколамский пр-д, влд. 7, корп.1",55.8007,37.4913,PROCESSING
4,"1-й Волоколамский проезд, дом 7, корпус 1",55.8008,37.4918,OLD
...,...,...,...,...
5960,"Ярцевская улица, дом 5",55.7356,37.4066,OLD
5961,"Ярцевская улица, дом 7",55.7357,37.4076,OLD
5962,"Ярцевская улица, дом 9",55.7363,37.4076,OLD
5963,"Яснополянская улица, дом 3, корпус 1",55.7222,37.7876,OLD


In [119]:
df_ren['status_code'].value_counts()

status_code
OLD           5176
FINISHED       340
START          219
PROCESSING     207
KVARTAL         23
Name: count, dtype: int64

In [120]:
#Сохраняем в Excel
writer = pd.ExcelWriter('renovation_m.xlsx')
df_ren.to_excel(writer, 'renovation_m')
writer._save()

In [95]:
import json

data = response.json()

# with open("renovation_nsk.json", "w", encoding = "utf-8") as f:
#     json.dump(data, f, indent = 4, ensure_ascii = False) # ensure_ascii=False для вывода кириллицы
with open("renovation_nsk.json", "w") as f:
    json.dump(data, f, indent = 4)  # indent=4 для красивого форматирования

# Варианты исключения нулей

In [57]:
number_str = "025:028:010022:0720"

parts = number_str.split(":")
cleaned_parts = [part.lstrip("0") for part in parts]
cleaned_number = ":".join(cleaned_parts)

print(cleaned_number)  # Вывод: 25:28:10022:720

25:28:10022:720


In [59]:
parts

['025', '028', '010022', '0720']

In [50]:
import re

number_str = "025:028:010022:0720"
cleaned_number = re.sub(r"0+(\d+)", r"\1", number_str)
print(cleaned_number)  # Вывод: 25:28:10022:720

25:28:10022:720


# Пример функции

In [ ]:
def get_data_from_url(url):
    """
    Получает данные JSON с указанного URL.

    Args:
        url: URL веб-страницы, содержащей данные в формате JSON.

    Returns:
        Словарь с данными, или None в случае ошибки.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Проверка на ошибки HTTP (4xx или 5xx)
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при получении данных с URL {url}: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Ошибка декодирования JSON с URL {url}: {e}")
        return None


def extract_address(data):
    """
    Извлекает адрес из словаря данных.

    Args:
        data: Словарь с данными.

    Returns:
        Адрес объекта, или None, если ключ "address" отсутствует или данные некорректны.
    """
    try:
        return data['address']
    except (KeyError, TypeError):
        return None


def main():
    urls = [
        "URL1",  # Замените на ваши URL-адреса
        "URL2",
        "URL3",
    ]

    addresses = {}
    for url in urls:
        data = get_data_from_url(url)
        if data:
            address = extract_address(data)
            if address:
                addresses[url] = address
            else:
                print(f"Адрес не найден для URL: {url}")

    print("\nАдреса объектов:")
    for url, address in addresses.items():
        print(f"URL: {url}, Адрес: {address}")


if __name__ == "__main__":
    main()